In [ ]:
# 保存生成器和判别器的權重
#generator.save_weights('generator_weights.h5')
#discriminator.save_weights('discriminator_weights.h5')

# 灰度轉換

In [14]:
# pip install opencv-python
# pip install opencv-python --upgrade

import cv2
import os

# 指定原始彩色圖像文件夾和目標灰度圖像文件夾
# original_folder = r'C:\Users\Tosha.E.T\Documents\GitHub\DolphinSkull\2022-3-8'
# gray_folder = r'C:\Users\Tosha.E.T\Documents\GitHub\DolphinSkull\2022-3-8\gray\class_1'
original_folder = r'C:\Users\Tosha.E.T\Documents\GitHub\DolphinSkull\number'
gray_folder = r'C:\Users\Tosha.E.T\Documents\GitHub\DolphinSkull\number\gray\class_1'

# 確保目標文件夾存在
if not os.path.exists(gray_folder):
    os.makedirs(gray_folder)

# 遍歷原始文件夾中的每個圖像
for filename in os.listdir(original_folder):
    if (filename.endswith('.JPG') or filename.endswith('.jpg')):
        # 讀取原始圖像
        original_image_path = os.path.join(original_folder, filename)
        original_image = cv2.imread(original_image_path)

        # 將彩色圖像轉換為灰度圖像
        gray_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2GRAY)

        # 保存灰度圖像到目標文件夾
        gray_image_path = os.path.join(gray_folder, filename)
        cv2.imwrite(gray_image_path, gray_image)

print("灰度轉換完成")


灰度轉換完成


# 普通的GAN

In [ ]:
import os
import numpy as np
import cv2 
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, PReLU
from keras.layers import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import winsound

print("start")

plt.switch_backend('agg')

noise_dim = 100
w=200
h=200

class GAN(object):
    """ Generative Adversarial Network class """
    def __init__(self, width=w, height=h, channels=1):

        self.width = width
        self.height = height
        self.channels = channels

        self.shape = (self.width, self.height, self.channels)
        
        self.optimizer = Adam(lr=0.0002, beta_1=0.6, decay=8e-8)
        self.optimizerG = Adam(lr=0.0002, beta_1=0.6, decay=8e-8)
        self.optimizerD = Adam(lr=0.0003, beta_1=0.6, decay=8e-8)

        self.G = self.__generator()
        self.G.compile(loss='binary_crossentropy', optimizer=self.optimizerG)

        self.D = self.__discriminator()
        self.D.compile(loss='binary_crossentropy', optimizer=self.optimizerD, metrics=['accuracy'])

        self.stacked_generator_discriminator = self.__stacked_generator_discriminator()

        self.stacked_generator_discriminator.compile(loss='binary_crossentropy', optimizer=self.optimizer)


    def __generator(self):
        """ Declare generator """

        model = Sequential()
        model.add(Dense(128, input_shape=(noise_dim,))) #noise_dim
        model.add(LeakyReLU(alpha=0.2)) #LeakyReLU(alpha=0.2)  #PReLU()
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(self.width  * self.height * self.channels, activation='tanh'))
        model.add(Reshape((self.width, self.height, self.channels)))

        return model

    def __discriminator(self):
        """ Declare discriminator """

        model = Sequential()
        model.add(Flatten(input_shape=self.shape))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(128))
        model.add(LeakyReLU(alpha=0.2))

        
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        return model

    def __stacked_generator_discriminator(self):

        self.D.trainable = False

        model = Sequential()
        model.add(self.G)
        model.add(self.D)

        return model
    
    
    def train(self, X_train, epochs=100000, batch=32, save_interval=1000):
        for cnt in range(epochs+1):

            # train discriminator
            random_index = np.random.randint(0, len(X_train) - np.int64(batch/2))
            legit_images = X_train[random_index : random_index + np.int64(batch/2)]

            gen_noise = np.random.normal(0, 1, (np.int64(batch/2), noise_dim)) #noise_dim
            syntetic_images = self.G.predict(gen_noise)

            x_combined_batch = np.concatenate((legit_images, syntetic_images))
            y_combined_batch = np.concatenate((np.ones((np.int64(batch/2), 1)), np.zeros((np.int64(batch/2), 1))))

            d_loss = self.D.train_on_batch(x_combined_batch, y_combined_batch)

            # train generator
            noise = np.random.normal(0, 1, (batch, noise_dim)) #noise_dim
            y_mislabled = np.ones((batch, 1))
            g_loss = self.stacked_generator_discriminator.train_on_batch(noise, y_mislabled)

            print ('epoch: %d, [Discriminator :: d_loss: %f], [ Generator :: loss: %f]' % (cnt, d_loss[0], g_loss))

            if cnt % save_interval == 0:
                self.plot_images(save2file=True, step=cnt)


    def plot_images(self, save2file=False, samples=16, step=0):
        if not os.path.exists("./images"):
            os.makedirs("./images")
        filename = "./images/dolphin_%d.png" % step
        noise = np.random.normal(0, 1, (samples, noise_dim)) #noise_dim

        images = self.G.predict(noise)

        plt.figure(figsize=(w, h))

        for i in range(images.shape[0]):
            plt.subplot(4, 4, i+1)
            image = images[i, :, :, 0]  # 取出灰階通道
            plt.imshow(image, cmap='gray')
            plt.axis('off')
        plt.tight_layout()

        if save2file:
            plt.savefig(filename)
            plt.close('all')
            winsound.Beep(1000,1000)
        else:
            plt.show()
    



if __name__ == '__main__':
    image_folder_path = r'2022-3-8\gray\class'
                        #r'C:\Users\Tosha.E.T\Documents\GitHub\DolphinSkull\number\gray\class_1'  
    image_files = os.listdir(image_folder_path)
    X_train = []

    for image_file in image_files:
        image_path = os.path.join(image_folder_path, image_file)
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

        if image is not None and not image.size == 0:
            # 如果圖片成功讀取，並且尺寸不為空，則進行調整大小
            image = cv2.resize(image, (w, h))
            X_train.append(image)
            #print(image)
        else:
            print(image_file)

    X_train = np.array(X_train) / 255.0
    X_train = np.expand_dims(X_train, axis=-1)

    gan = GAN()
    gan.train(X_train)
    

    
# 訓練完畢提示音    
winsound.Beep(2000,2000)

start


c:\users\tosha.e.t\appdata\local\programs\python\python39\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 40000)             0         
                                                                 
 dense_5 (Dense)             (None, 1024)              40961024  
                                                                 
 leaky_re_lu_4 (LeakyReLU)   (None, 1024)              0         
                                                                 
 dense_6 (Dense)             (None, 512)               524800    
                                                                 
 leaky_re_lu_5 (LeakyReLU)   (None, 512)               0         
                                                                 
 dense_7 (Dense)             (None, 256)               131328    
                                                                 
 leaky_re_lu_6 (LeakyReLU)   (None, 256)              

epoch: 82, [Discriminator :: d_loss: 0.277875], [ Generator :: loss: 12.061771]
epoch: 83, [Discriminator :: d_loss: 0.058093], [ Generator :: loss: 12.815023]
epoch: 84, [Discriminator :: d_loss: 0.014964], [ Generator :: loss: 9.109312]
epoch: 85, [Discriminator :: d_loss: 0.014350], [ Generator :: loss: 12.277335]
epoch: 86, [Discriminator :: d_loss: 0.025714], [ Generator :: loss: 10.157678]
epoch: 87, [Discriminator :: d_loss: 0.021571], [ Generator :: loss: 9.422870]
epoch: 88, [Discriminator :: d_loss: 0.007269], [ Generator :: loss: 10.353374]
epoch: 89, [Discriminator :: d_loss: 0.022592], [ Generator :: loss: 11.000423]
epoch: 90, [Discriminator :: d_loss: 0.066125], [ Generator :: loss: 7.876807]
epoch: 91, [Discriminator :: d_loss: 0.060051], [ Generator :: loss: 10.094358]
epoch: 92, [Discriminator :: d_loss: 0.027285], [ Generator :: loss: 10.568388]
epoch: 93, [Discriminator :: d_loss: 0.138946], [ Generator :: loss: 10.794901]
epoch: 94, [Discriminator :: d_loss: 0.0696

epoch: 184, [Discriminator :: d_loss: 0.058648], [ Generator :: loss: 17.380003]
epoch: 185, [Discriminator :: d_loss: 0.053091], [ Generator :: loss: 14.094132]
epoch: 186, [Discriminator :: d_loss: 0.012782], [ Generator :: loss: 12.975370]
epoch: 187, [Discriminator :: d_loss: 0.009276], [ Generator :: loss: 10.006459]
epoch: 188, [Discriminator :: d_loss: 0.016898], [ Generator :: loss: 11.215294]
epoch: 189, [Discriminator :: d_loss: 0.007166], [ Generator :: loss: 15.212299]
epoch: 190, [Discriminator :: d_loss: 0.038655], [ Generator :: loss: 12.084194]
epoch: 191, [Discriminator :: d_loss: 0.024941], [ Generator :: loss: 11.217115]
epoch: 192, [Discriminator :: d_loss: 0.060115], [ Generator :: loss: 11.044460]
epoch: 193, [Discriminator :: d_loss: 0.070819], [ Generator :: loss: 13.351398]
epoch: 194, [Discriminator :: d_loss: 0.147106], [ Generator :: loss: 10.701869]
epoch: 195, [Discriminator :: d_loss: 0.162621], [ Generator :: loss: 8.617199]
epoch: 196, [Discriminator ::

...

epoch: 22625, [Discriminator :: d_loss: 0.163429], [ Generator :: loss: 4.709844]
epoch: 22626, [Discriminator :: d_loss: 0.299976], [ Generator :: loss: 3.433061]
epoch: 22627, [Discriminator :: d_loss: 0.045647], [ Generator :: loss: 4.383911]
epoch: 22628, [Discriminator :: d_loss: 0.061428], [ Generator :: loss: 4.870496]
epoch: 22629, [Discriminator :: d_loss: 0.023197], [ Generator :: loss: 5.504164]
epoch: 22630, [Discriminator :: d_loss: 0.016297], [ Generator :: loss: 5.574862]
epoch: 22631, [Discriminator :: d_loss: 0.012648], [ Generator :: loss: 5.726042]
epoch: 22632, [Discriminator :: d_loss: 0.367946], [ Generator :: loss: 4.851205]
epoch: 22633, [Discriminator :: d_loss: 0.046754], [ Generator :: loss: 3.916394]
epoch: 22634, [Discriminator :: d_loss: 0.046017], [ Generator :: loss: 4.690317]
epoch: 22635, [Discriminator :: d_loss: 0.220917], [ Generator :: loss: 3.788264]
epoch: 22636, [Discriminator :: d_loss: 0.305169], [ Generator :: loss: 3.084513]
epoch: 22637, [D

epoch: 22725, [Discriminator :: d_loss: 0.043781], [ Generator :: loss: 5.179327]
epoch: 22726, [Discriminator :: d_loss: 0.010892], [ Generator :: loss: 5.298136]
epoch: 22727, [Discriminator :: d_loss: 0.004222], [ Generator :: loss: 6.848173]
epoch: 22728, [Discriminator :: d_loss: 0.001388], [ Generator :: loss: 8.364986]
epoch: 22729, [Discriminator :: d_loss: 0.158909], [ Generator :: loss: 9.295792]
epoch: 22730, [Discriminator :: d_loss: 0.003186], [ Generator :: loss: 10.769276]
epoch: 22731, [Discriminator :: d_loss: 0.060119], [ Generator :: loss: 12.413774]
epoch: 22732, [Discriminator :: d_loss: 0.005729], [ Generator :: loss: 6.496197]
epoch: 22733, [Discriminator :: d_loss: 0.005904], [ Generator :: loss: 11.365684]
epoch: 22734, [Discriminator :: d_loss: 0.001586], [ Generator :: loss: 7.717515]
epoch: 22735, [Discriminator :: d_loss: 0.000908], [ Generator :: loss: 6.665692]
epoch: 22736, [Discriminator :: d_loss: 0.536101], [ Generator :: loss: 4.494499]
epoch: 22737,

epoch: 22825, [Discriminator :: d_loss: 0.020586], [ Generator :: loss: 4.587913]
epoch: 22826, [Discriminator :: d_loss: 0.093929], [ Generator :: loss: 5.320003]
epoch: 22827, [Discriminator :: d_loss: 0.021744], [ Generator :: loss: 5.164407]
epoch: 22828, [Discriminator :: d_loss: 0.235826], [ Generator :: loss: 5.023819]
epoch: 22829, [Discriminator :: d_loss: 0.110582], [ Generator :: loss: 5.931905]
epoch: 22830, [Discriminator :: d_loss: 0.008007], [ Generator :: loss: 6.417688]
epoch: 22831, [Discriminator :: d_loss: 0.001930], [ Generator :: loss: 6.823143]
epoch: 22832, [Discriminator :: d_loss: 0.696008], [ Generator :: loss: 4.355129]
epoch: 22833, [Discriminator :: d_loss: 0.015976], [ Generator :: loss: 3.343349]
epoch: 22834, [Discriminator :: d_loss: 0.061833], [ Generator :: loss: 4.198073]
epoch: 22835, [Discriminator :: d_loss: 0.314812], [ Generator :: loss: 2.772425]
epoch: 22836, [Discriminator :: d_loss: 0.286155], [ Generator :: loss: 2.068060]
epoch: 22837, [D

epoch: 22925, [Discriminator :: d_loss: 0.145428], [ Generator :: loss: 5.293064]
epoch: 22926, [Discriminator :: d_loss: 0.047577], [ Generator :: loss: 5.959697]
epoch: 22927, [Discriminator :: d_loss: 0.026242], [ Generator :: loss: 6.347402]
epoch: 22928, [Discriminator :: d_loss: 0.209357], [ Generator :: loss: 5.002524]
epoch: 22929, [Discriminator :: d_loss: 0.035440], [ Generator :: loss: 5.143045]
epoch: 22930, [Discriminator :: d_loss: 0.020290], [ Generator :: loss: 5.090831]
epoch: 22931, [Discriminator :: d_loss: 0.038604], [ Generator :: loss: 6.378303]
epoch: 22932, [Discriminator :: d_loss: 0.011245], [ Generator :: loss: 6.336441]
epoch: 22933, [Discriminator :: d_loss: 0.016381], [ Generator :: loss: 6.976151]
epoch: 22934, [Discriminator :: d_loss: 0.106671], [ Generator :: loss: 6.603540]
epoch: 22935, [Discriminator :: d_loss: 0.007297], [ Generator :: loss: 8.077667]
epoch: 22936, [Discriminator :: d_loss: 0.003568], [ Generator :: loss: 6.821527]
epoch: 22937, [D

epoch: 23025, [Discriminator :: d_loss: 0.021287], [ Generator :: loss: 5.886217]
epoch: 23026, [Discriminator :: d_loss: 0.046980], [ Generator :: loss: 6.907324]
epoch: 23027, [Discriminator :: d_loss: 0.286483], [ Generator :: loss: 5.117407]
epoch: 23028, [Discriminator :: d_loss: 0.056794], [ Generator :: loss: 4.258047]
epoch: 23029, [Discriminator :: d_loss: 0.054259], [ Generator :: loss: 4.791416]
epoch: 23030, [Discriminator :: d_loss: 0.040908], [ Generator :: loss: 4.384789]
epoch: 23031, [Discriminator :: d_loss: 0.074192], [ Generator :: loss: 5.918446]
epoch: 23032, [Discriminator :: d_loss: 0.278672], [ Generator :: loss: 5.480372]
epoch: 23033, [Discriminator :: d_loss: 0.056325], [ Generator :: loss: 5.552470]
epoch: 23034, [Discriminator :: d_loss: 0.150948], [ Generator :: loss: 3.863316]
epoch: 23035, [Discriminator :: d_loss: 0.043043], [ Generator :: loss: 3.817756]
epoch: 23036, [Discriminator :: d_loss: 0.057323], [ Generator :: loss: 5.576019]
epoch: 23037, [D

# DCGAN

In [7]:
import os
import numpy as np
import cv2 
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, LeakyReLU
from keras.layers import BatchNormalization, Conv2D, Conv2DTranspose
from keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import winsound

print("start")

plt.switch_backend('agg')

noise_dim = 100
w = 200
h = 200

class DCGAN(object):
    """ Deep Convolutional Generative Adversarial Network class """
    def __init__(self, width=w, height=h, channels=1):

        self.width = width
        self.height = height
        self.channels = channels

        self.shape = (self.width, self.height, self.channels)
        
        self.optimizer = Adam(lr=0.0002, beta_1=0.6, decay=8e-8)

        self.G = self.__generator()
        self.G.compile(loss='binary_crossentropy', optimizer=self.optimizer)

        self.D = self.__discriminator()
        self.D.compile(loss='binary_crossentropy', optimizer=self.optimizer, metrics=['accuracy'])

        self.stacked_generator_discriminator = self.__stacked_generator_discriminator()

        self.stacked_generator_discriminator.compile(loss='binary_crossentropy', optimizer=self.optimizer)


    def __generator(self):
        """ Declare generator """

        model = Sequential()

        model.add(Dense(128 * 25 * 25, activation="relu", input_dim=noise_dim))
        model.add(Reshape((25, 25, 128)))
        model.add(BatchNormalization(momentum=0.8))
        
        model.add(Conv2DTranspose(128, kernel_size=3, strides=2, padding="same", activation="relu"))
        model.add(BatchNormalization(momentum=0.8))
        
        model.add(Conv2DTranspose(64, kernel_size=3, strides=2, padding="same", activation="relu"))
        model.add(BatchNormalization(momentum=0.8))
        
        model.add(Conv2DTranspose(1, kernel_size=3, strides=1, padding="same", activation="tanh"))

        return model

    def __discriminator(self):
        """ Declare discriminator """

        model = Sequential()

        model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=self.shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        
        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))

        return model
    
    def __stacked_generator_discriminator(self):
        self.D.trainable = False
        noise = Input(shape=(noise_dim,))
        img = self.G(noise)
        valid = self.D(img)
        #valid = self.D(img_flat)
        return Model(noise, valid)

    
    
    def train(self, X_train, epochs=2000, batch=32, save_interval=100):
        for cnt in range(epochs):

            # train discriminator
            random_index = np.random.randint(0, len(X_train) - np.int64(batch/2))
            legit_images = X_train[random_index : random_index + np.int64(batch/2)]

            gen_noise = np.random.normal(0, 1, (np.int64(batch/2), noise_dim))
            syntetic_images = self.G.predict(gen_noise)

            x_combined_batch = np.concatenate((legit_images, syntetic_images))
            y_combined_batch = np.concatenate((np.ones((np.int64(batch/2), 1)), np.zeros((np.int64(batch/2), 1))))

            d_loss = self.D.train_on_batch(x_combined_batch, y_combined_batch)

            # train generator
            noise = np.random.normal(0, 1, (batch, noise_dim))
            y_mislabled = np.ones((batch, 1))
            g_loss = self.stacked_generator_discriminator.train_on_batch(noise, y_mislabled)

            print ('epoch: %d, [Discriminator :: d_loss: %f], [ Generator :: loss: %f]' % (cnt, d_loss[0], g_loss))

            if cnt % save_interval == 0:
                self.plot_images(save2file=True, step=cnt)


    def plot_images(self, save2file=False, samples=16, step=0):
        if not os.path.exists("./images"):
            os.makedirs("./images")
        filename = "./images/DCGAN/dolphin_%d.png" % step
        noise = np.random.normal(0, 1, (samples, noise_dim))

        images = self.G.predict(noise)

        plt.figure(figsize=(w, h))

        for i in range(images.shape[0]):
            plt.subplot(4, 4, i+1)
            image = images[i, :, :, 0]
            plt.imshow(image, cmap='gray')
            plt.axis('off')
        plt.tight_layout()

        if save2file:
            plt.savefig(filename)
            plt.close('all')
            winsound.Beep(1000,1000)
        else:
            plt.show()
    

if __name__ == '__main__':
    image_folder_path = r'2022-3-8\gray\class'
    image_files = os.listdir(image_folder_path)
    X_train = []

    for image_file in image_files:
        image_path = os.path.join(image_folder_path, image_file)
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

        if image is not None and not image.size == 0:
            image = cv2.resize(image, (w, h))
            X_train.append(image)
        else:
            print(image_file)

    X_train = np.array(X_train) / 255.0
    X_train = np.expand_dims(X_train, axis=-1)

    dcgan = DCGAN()
    dcgan.train(X_train)
    
# 訓練完畢提示音    
winsound.Beep(2000,2000)


start


ValueError: Exception encountered when calling layer "sequential_13" (type Sequential).

Input 0 of layer "dense_11" is incompatible with the layer: expected axis -1of input shape to have value 80000, but received input with shape (None, 21632)

Call arguments received:
  • inputs=tf.Tensor(shape=(None, 100, 100, 1), dtype=float32)
  • training=None
  • mask=None

In [1]:
print("gsdgf")

gsdgf


# WGAN

In [16]:
import os
import numpy as np
import cv2 
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, PReLU
from keras.layers import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential
from keras import backend as K
from tensorflow.keras.optimizers import Adam, RMSprop
import matplotlib.pyplot as plt
import winsound

plt.switch_backend('agg')

noise_dim = 200

class WGAN(object):
    """ Wasserstein Generative Adversarial Network class """
    def __init__(self, width=50, height=50, channels=1):

        self.width = width
        self.height = height
        self.channels = channels

        self.shape = (self.width, self.height, self.channels)
        
        self.optimizer = RMSprop(lr=0.00005)

        self.G = self.__generator()
        self.G.compile(loss=self.wasserstein_loss, optimizer=self.optimizer)  # 修正此處

        self.D = self.__discriminator()
        self.D.compile(loss=self.wasserstein_loss, optimizer=self.optimizer)  # 修正此處

        self.stacked_generator_discriminator = self.__stacked_generator_discriminator()

        self.stacked_generator_discriminator.compile(loss=self.wasserstein_loss, optimizer=self.optimizer)  # 修正此處
        
    
    def wasserstein_loss(self, y_true, y_pred):
        return K.mean(y_true * y_pred)


    def __generator(self):
        """ Declare generator """

        model = Sequential()
        model.add(Dense(256, input_shape=(200,))) #noise_dim
        model.add(PReLU()) #LeakyReLU(alpha=0.2)
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(PReLU())
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(PReLU())
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(2048))
        model.add(PReLU())
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(4096))
        model.add(PReLU())
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(8192))
        model.add(PReLU())
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(self.width  * self.height * self.channels, activation='tanh'))
        model.add(Reshape((self.width, self.height, self.channels)))

        return model

    def __discriminator(self):
        """ Declare discriminator """

        model = Sequential()
        model.add(Flatten(input_shape=self.shape))
        model.add(Dense(8192))
        model.add(PReLU())
        model.add(Dense(4096))
        model.add(PReLU())
        model.add(Dense(2048))
        model.add(PReLU())
        model.add(Dense(1024))
        model.add(PReLU())
        model.add(Dense(512))
        model.add(PReLU())
        model.add(Dense(256))
        model.add(PReLU())
        
        # 使用線性激活函數而不是 sigmoid
        model.add(Dense(1, activation='linear'))
        
        model.summary()

        return model

    def __stacked_generator_discriminator(self):

        self.D.trainable = False

        model = Sequential()
        model.add(self.G)
        model.add(self.D)

        return model
    
    #epochs=20000
    def train(self, X_train, epochs=100002, batch=32, save_interval=100, clip_value=0.01):
        for cnt in range(epochs):

            # train discriminator
            random_index = np.random.randint(0, len(X_train) - np.int64(batch/2))
            legit_images = X_train[random_index : random_index + np.int64(batch/2)]

            gen_noise = np.random.normal(0, 1, (np.int64(batch/2), noise_dim))
            syntetic_images = self.G.predict(gen_noise)

            x_combined_batch = np.concatenate((legit_images, syntetic_images))
            y_combined_batch = np.concatenate((-np.ones((np.int64(batch/2), 1)), np.ones((np.int64(batch/2), 1))))

            d_loss = self.D.train_on_batch(x_combined_batch, y_combined_batch)
            # Clip discriminator weights
            for l in self.D.layers:
                weights = l.get_weights()
                weights = [np.clip(w, -clip_value, clip_value) for w in weights]
                l.set_weights(weights)

            # train generator
            noise = np.random.normal(0, 1, (batch, noise_dim))
            y_mislabled = -np.ones((batch, 1))
            g_loss = self.stacked_generator_discriminator.train_on_batch(noise, y_mislabled)

            print ('epoch: %d, [Discriminator :: d_loss: %f], [ Generator :: loss: %f]' % (cnt, -d_loss, -g_loss))

            if cnt % save_interval == 0:
                self.plot_images(save2file=True, step=cnt)


    def plot_images(self, save2file=False, samples=16, step=0):
        if not os.path.exists("./images"):
            os.makedirs("./images")
        filename = "./images/WGAN/Dolphin_%d.png" % step
        noise = np.random.normal(0, 1, (samples, noise_dim)) #noise_dim

        images = self.G.predict(noise)

        plt.figure(figsize=(50, 50))

        for i in range(images.shape[0]):
            plt.subplot(4, 4, i+1)
            image = images[i, :, :, 0]  # 取出灰階通道
            plt.imshow(image, cmap='gray')
            plt.axis('off')
        plt.tight_layout()

        if save2file:
            plt.savefig(filename)
            plt.close('all')
            winsound.Beep(1000,1000)
        else:
            plt.show()
    



if __name__ == '__main__':
    image_folder_path = r'C:\Users\Tosha.E.T\Documents\GitHub\DolphinSkull\2022-3-8\gray\class' #r'C:\Users\Tosha.E.T\Documents\GitHub\DolphinSkull\number\gray\class_1'#
    image_files = os.listdir(image_folder_path)
    X_train = []

    for image_file in image_files:
        image_path = os.path.join(image_folder_path, image_file)
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

        if image is not None and not image.size == 0:
            # 如果圖片成功讀取，並且尺寸不為空，則進行調整大小
            image = cv2.resize(image, (50, 50))
            X_train.append(image)
            #print(image)
        else:
            print(image_file)

    X_train = np.array(X_train) / 255.0
    X_train = np.expand_dims(X_train, axis=-1)

    wgan = WGAN()
    wgan.train(X_train)
    

    
# 訓練完畢提示音    
winsound.Beep(2000,2000)

Model: "sequential_40"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_13 (Flatten)        (None, 2500)              0         
                                                                 
 dense_189 (Dense)           (None, 8192)              20488192  
                                                                 
 p_re_lu_162 (PReLU)         (None, 8192)              8192      
                                                                 
 dense_190 (Dense)           (None, 4096)              33558528  
                                                                 
 p_re_lu_163 (PReLU)         (None, 4096)              4096      
                                                                 
 dense_191 (Dense)           (None, 2048)              8390656   
                                                                 
 p_re_lu_164 (PReLU)         (None, 2048)            

epoch: 74, [Discriminator :: d_loss: 54.750816], [ Generator :: loss: -95.626495]
epoch: 75, [Discriminator :: d_loss: 59.889709], [ Generator :: loss: -100.886826]
epoch: 76, [Discriminator :: d_loss: 60.067444], [ Generator :: loss: -106.905746]
epoch: 77, [Discriminator :: d_loss: 58.852829], [ Generator :: loss: -113.332855]
epoch: 78, [Discriminator :: d_loss: 64.155167], [ Generator :: loss: -118.399918]
epoch: 79, [Discriminator :: d_loss: 70.154381], [ Generator :: loss: -126.317696]
epoch: 80, [Discriminator :: d_loss: 69.705933], [ Generator :: loss: -133.702087]
epoch: 81, [Discriminator :: d_loss: 76.107712], [ Generator :: loss: -139.101044]
epoch: 82, [Discriminator :: d_loss: 85.452888], [ Generator :: loss: -146.154816]
epoch: 83, [Discriminator :: d_loss: 85.121086], [ Generator :: loss: -155.520172]
epoch: 84, [Discriminator :: d_loss: 101.454857], [ Generator :: loss: -165.790558]
epoch: 85, [Discriminator :: d_loss: 102.154572], [ Generator :: loss: -172.280334]
epo

epoch: 171, [Discriminator :: d_loss: 1817.984131], [ Generator :: loss: -2623.865967]
epoch: 172, [Discriminator :: d_loss: 1674.933594], [ Generator :: loss: -2683.992432]
epoch: 173, [Discriminator :: d_loss: 1818.508789], [ Generator :: loss: -2721.616699]
epoch: 174, [Discriminator :: d_loss: 1778.414795], [ Generator :: loss: -2790.900391]
epoch: 175, [Discriminator :: d_loss: 2051.917969], [ Generator :: loss: -2829.719727]
epoch: 176, [Discriminator :: d_loss: 1891.021729], [ Generator :: loss: -2892.079102]
epoch: 177, [Discriminator :: d_loss: 1889.021973], [ Generator :: loss: -2996.929199]
epoch: 178, [Discriminator :: d_loss: 2096.328613], [ Generator :: loss: -3021.721191]
epoch: 179, [Discriminator :: d_loss: 2224.812988], [ Generator :: loss: -3051.632812]
epoch: 180, [Discriminator :: d_loss: 2288.182129], [ Generator :: loss: -3168.939453]
epoch: 181, [Discriminator :: d_loss: 2118.265381], [ Generator :: loss: -3196.854492]
epoch: 182, [Discriminator :: d_loss: 2226.

epoch: 266, [Discriminator :: d_loss: 9222.716797], [ Generator :: loss: -11221.218750]
epoch: 267, [Discriminator :: d_loss: 9609.536133], [ Generator :: loss: -11111.888672]
epoch: 268, [Discriminator :: d_loss: 10663.352539], [ Generator :: loss: -11254.248047]
epoch: 269, [Discriminator :: d_loss: 10224.057617], [ Generator :: loss: -11489.302734]
epoch: 270, [Discriminator :: d_loss: 11543.586914], [ Generator :: loss: -11629.652344]
epoch: 271, [Discriminator :: d_loss: 10108.441406], [ Generator :: loss: -11709.632812]
epoch: 272, [Discriminator :: d_loss: 8663.208984], [ Generator :: loss: -12127.894531]
epoch: 273, [Discriminator :: d_loss: 10949.512695], [ Generator :: loss: -11891.853516]
epoch: 274, [Discriminator :: d_loss: 9839.603516], [ Generator :: loss: -12243.994141]
epoch: 275, [Discriminator :: d_loss: 10228.441406], [ Generator :: loss: -12682.170898]
epoch: 276, [Discriminator :: d_loss: 10247.654297], [ Generator :: loss: -12014.109375]
epoch: 277, [Discriminato

epoch: 359, [Discriminator :: d_loss: 25559.705078], [ Generator :: loss: -22293.023438]
epoch: 360, [Discriminator :: d_loss: 17691.675781], [ Generator :: loss: -22940.535156]
epoch: 361, [Discriminator :: d_loss: 25349.160156], [ Generator :: loss: -22261.085938]
epoch: 362, [Discriminator :: d_loss: 29673.234375], [ Generator :: loss: -21351.460938]
epoch: 363, [Discriminator :: d_loss: 22087.414062], [ Generator :: loss: -21306.496094]
epoch: 364, [Discriminator :: d_loss: 22392.250000], [ Generator :: loss: -22374.312500]
epoch: 365, [Discriminator :: d_loss: 22850.203125], [ Generator :: loss: -23405.726562]
epoch: 366, [Discriminator :: d_loss: 25424.289062], [ Generator :: loss: -22683.015625]
epoch: 367, [Discriminator :: d_loss: 22322.125000], [ Generator :: loss: -22142.664062]
epoch: 368, [Discriminator :: d_loss: 28554.113281], [ Generator :: loss: -22899.458984]
epoch: 369, [Discriminator :: d_loss: 24715.636719], [ Generator :: loss: -21960.830078]
epoch: 370, [Discrimi

...

epoch: 18844, [Discriminator :: d_loss: 37298.226562], [ Generator :: loss: -46432.335938]
epoch: 18845, [Discriminator :: d_loss: 44741.468750], [ Generator :: loss: -61917.210938]
epoch: 18846, [Discriminator :: d_loss: 44623.179688], [ Generator :: loss: -16643.345703]
epoch: 18847, [Discriminator :: d_loss: 36714.144531], [ Generator :: loss: -51093.117188]
epoch: 18848, [Discriminator :: d_loss: 21370.861328], [ Generator :: loss: -23131.039062]
epoch: 18849, [Discriminator :: d_loss: 38086.261719], [ Generator :: loss: -46701.863281]
epoch: 18850, [Discriminator :: d_loss: 40662.117188], [ Generator :: loss: -43068.804688]
epoch: 18851, [Discriminator :: d_loss: 39169.277344], [ Generator :: loss: -29667.986328]
epoch: 18852, [Discriminator :: d_loss: 50482.480469], [ Generator :: loss: -58420.769531]
epoch: 18853, [Discriminator :: d_loss: 27618.134766], [ Generator :: loss: -31461.695312]
epoch: 18854, [Discriminator :: d_loss: 54676.992188], [ Generator :: loss: -92501.312500]

epoch: 18935, [Discriminator :: d_loss: 28083.224609], [ Generator :: loss: -7022.721680]
epoch: 18936, [Discriminator :: d_loss: 27441.789062], [ Generator :: loss: -23401.193359]
epoch: 18937, [Discriminator :: d_loss: 37579.062500], [ Generator :: loss: -66517.343750]
epoch: 18938, [Discriminator :: d_loss: 31648.808594], [ Generator :: loss: -25439.531250]
epoch: 18939, [Discriminator :: d_loss: 41153.859375], [ Generator :: loss: -39640.843750]
epoch: 18940, [Discriminator :: d_loss: 44016.238281], [ Generator :: loss: -14751.982422]
epoch: 18941, [Discriminator :: d_loss: 39927.335938], [ Generator :: loss: -46061.796875]
epoch: 18942, [Discriminator :: d_loss: 36621.003906], [ Generator :: loss: -78727.578125]
epoch: 18943, [Discriminator :: d_loss: 33488.414062], [ Generator :: loss: -11134.006836]
epoch: 18944, [Discriminator :: d_loss: 25726.921875], [ Generator :: loss: -48205.218750]
epoch: 18945, [Discriminator :: d_loss: 41740.523438], [ Generator :: loss: -25736.742188]


epoch: 19026, [Discriminator :: d_loss: 35872.156250], [ Generator :: loss: -18463.675781]
epoch: 19027, [Discriminator :: d_loss: 38177.023438], [ Generator :: loss: -44261.328125]
epoch: 19028, [Discriminator :: d_loss: 44599.171875], [ Generator :: loss: -55943.406250]
epoch: 19029, [Discriminator :: d_loss: 40984.160156], [ Generator :: loss: -40243.593750]
epoch: 19030, [Discriminator :: d_loss: 27874.517578], [ Generator :: loss: -8785.849609]
epoch: 19031, [Discriminator :: d_loss: 40999.406250], [ Generator :: loss: -40926.281250]
epoch: 19032, [Discriminator :: d_loss: 38919.859375], [ Generator :: loss: -47812.261719]
epoch: 19033, [Discriminator :: d_loss: 30144.957031], [ Generator :: loss: -44112.746094]
epoch: 19034, [Discriminator :: d_loss: 39959.457031], [ Generator :: loss: -43005.242188]
epoch: 19035, [Discriminator :: d_loss: 30810.191406], [ Generator :: loss: -49521.164062]
epoch: 19036, [Discriminator :: d_loss: 69375.476562], [ Generator :: loss: -60179.316406]


epoch: 19117, [Discriminator :: d_loss: 56700.644531], [ Generator :: loss: -65976.476562]
epoch: 19118, [Discriminator :: d_loss: 34301.687500], [ Generator :: loss: -25348.859375]
epoch: 19119, [Discriminator :: d_loss: 37153.210938], [ Generator :: loss: -27227.445312]
epoch: 19120, [Discriminator :: d_loss: 43519.289062], [ Generator :: loss: -49180.812500]
epoch: 19121, [Discriminator :: d_loss: 25450.671875], [ Generator :: loss: -24648.841797]
epoch: 19122, [Discriminator :: d_loss: 38016.867188], [ Generator :: loss: -30623.394531]
epoch: 19123, [Discriminator :: d_loss: 46382.894531], [ Generator :: loss: -55794.339844]
epoch: 19124, [Discriminator :: d_loss: 35664.375000], [ Generator :: loss: -26863.109375]
epoch: 19125, [Discriminator :: d_loss: 56116.906250], [ Generator :: loss: -42305.656250]
epoch: 19126, [Discriminator :: d_loss: 25502.250000], [ Generator :: loss: -59588.000000]
epoch: 19127, [Discriminator :: d_loss: 46834.699219], [ Generator :: loss: -59007.578125]

epoch: 19208, [Discriminator :: d_loss: 41389.785156], [ Generator :: loss: -46994.335938]
epoch: 19209, [Discriminator :: d_loss: 47349.328125], [ Generator :: loss: -26585.578125]
epoch: 19210, [Discriminator :: d_loss: 39224.890625], [ Generator :: loss: -35692.273438]
epoch: 19211, [Discriminator :: d_loss: 35738.644531], [ Generator :: loss: -11909.750000]
epoch: 19212, [Discriminator :: d_loss: 24456.039062], [ Generator :: loss: -24563.144531]
epoch: 19213, [Discriminator :: d_loss: 40472.824219], [ Generator :: loss: -33037.312500]
epoch: 19214, [Discriminator :: d_loss: 49710.457031], [ Generator :: loss: -66416.742188]
epoch: 19215, [Discriminator :: d_loss: 36094.464844], [ Generator :: loss: -9474.873047]
epoch: 19216, [Discriminator :: d_loss: 37391.410156], [ Generator :: loss: -20643.671875]
epoch: 19217, [Discriminator :: d_loss: 50421.589844], [ Generator :: loss: -58665.992188]
epoch: 19218, [Discriminator :: d_loss: 22180.429688], [ Generator :: loss: -38360.835938]
